In [ ]:
# import asyncio
from eunis_client import EunisClient

# 1. Define the target service
SERVICE_NAME = "EUNIS/Grassland_Distribution_point"

async def run_analysis():
    # Context manager handles the session cleanup automatically
    async with EunisClient() as client:
        print(f"Connecting to {client.base_url}...")

        # A. Quick Preview (Fast)
        print("Fetching preview...")
        preview_df = await client.get_preview(SERVICE_NAME, limit=500)

        # Display map (if ipyleaflet/folium installed) or simple plot
        print(f"Preview rows: {len(preview_df)}")
        if not preview_df.empty:
            preview_df.plot()

        # B. Full Download (Slower, rigorous)
        print("\nStarting full download...")
        try:
            full_df = await client.download_layer(SERVICE_NAME)
            print(f"Successfully downloaded {len(full_df)} records.")

            # Save to disk
            full_df.to_file("grasslands.gpkg", driver="GPKG")
            print("Saved to grasslands.gpkg")

        except Exception as e:
            print(f"Download failed: {e}")

# Run the async function in the notebook loop
await run_analysis()